# Construção das janelas de 5 partidas (`team_windows_hist5`)

Este notebook constrói vetores de atributos baseados em **janelas históricas de 5 partidas**
para cada time.

Cada linha da tabela `team_windows_hist5` representa:

- um **time** (`team_id`);
- uma **partida atual** (`current_match_id`, `current_date`);
- o **resultado da partida atual** (`result_current = -1, 0, 1`);
- uma janela com as **5 partidas anteriores** desse time, ordenadas do histórico mais recente
  para o mais antigo.

Para cada uma das 5 partidas anteriores são usados os seguintes atributos:

- `goals_for`, `goals_against`
- `shots_for`, `shots_against`
- `shots_on_for`, `shots_on_against`
- `possession_for`, `possession_against`
- `corners_for`, `corners_against`
- `crosses_for`, `crosses_against`
- `fouls_for`, `fouls_against`
- `yellows_for`, `yellows_against`
- `reds_for`, `reds_against`
- `is_home`
- `result_code` (resultado daquela partida histórica)

Total: **20 atributos por partida × 5 partidas = 100 features** de histórico por janela.

O resultado é salvo na tabela `team_windows_hist5` no banco SQLite.


In [ ]:
# 1. Imports, caminho do banco e definição da janela

import sqlite3
import pandas as pd
import numpy as np

DB_PATH = "../data/database.sqlite"
WINDOW_SIZE = 5

# Atributos por partida do ponto de vista do time
base_feature_cols = [
    "goals_for", "goals_against",
    "shots_for", "shots_against",
    "shots_on_for", "shots_on_against",
    "possession_for", "possession_against",
    "corners_for", "corners_against",
    "crosses_for", "crosses_against",
    "fouls_for", "fouls_against",
    "yellows_for", "yellows_against",
    "reds_for", "reds_against",
    "is_home",
    "result_code",  # resultado da partida histórica
]


In [ ]:
# 2. Função auxiliar: gera nomes das colunas de histórico para cada posição da janela

def build_window_feature_names(window_size: int, cols: list[str]) -> list[str]:
    """
    Gera nomes do tipo <feature>_k, onde:
    - k = 1: partida MAIS recente do histórico (logo antes da partida atual)
    - k = window_size: partida mais antiga da janela

    Ex.: goals_for_1, goals_for_2, ..., goals_for_5
    """
    names = []
    for k in range(1, window_size + 1):
        for col in cols:
            names.append(f"{col}_{k}")
    return names


In [ ]:
# 3. Carregar a tabela base por time e partida

conn = sqlite3.connect(DB_PATH)
df_tm = pd.read_sql_query("SELECT * FROM team_match_features;", conn)

print("team_match_features:", df_tm.shape)
df_tm.head()


In [ ]:
# 4. Preparar tipos e tratar valores ausentes

# Converter coluna de data
df_tm["date"] = pd.to_datetime(df_tm["date"], errors="coerce")

# Preencher NaNs nos atributos numéricos da base
df_tm[base_feature_cols] = df_tm[base_feature_cols].fillna(0.0)

# Gerar nomes das features da janela
window_feature_names = build_window_feature_names(WINDOW_SIZE, base_feature_cols)

print("Quantidade de features por janela:", len(window_feature_names))
print("Exemplos de nomes:", window_feature_names[:10])


## Geração das janelas históricas (5 partidas anteriores)

Para cada time (`team_id`):

1. Ordenamos as partidas em ordem crescente de data.
2. Só consideramos times com **mais de 5 partidas** (senão não é possível formar janela de 5 jogos).
3. Para cada índice `i` a partir de `WINDOW_SIZE`:
   - Consideramos a partida atual como `i`;
   - Construímos o histórico com as 5 partidas anteriores: índices `[i-5, ..., i-1]`;
   - Reordenamos esse histórico internamente para que:
     - sufixo `_1` = partida mais recente do histórico (i-1),
     - sufixo `_5` = partida mais antiga (i-5).
4. Empilhamos todos os vetores de histórico em `X` e os metadados em `df_context`.


In [ ]:
# 5. Geração das janelas históricas de tamanho 5

all_windows = []
all_contexts = []

for team_id, df_team in df_tm.groupby("team_id"):
    # Ordena as partidas desse time por data
    df_team = df_team.sort_values("date").reset_index(drop=True)
    n = len(df_team)

    # É necessário ter pelo menos 5 partidas ANTERIORES + 1 atual
    if n <= WINDOW_SIZE:
        continue

    # i = índice da PARTIDA ATUAL
    # janelas vão de i = WINDOW_SIZE até n-1
    for i in range(WINDOW_SIZE, n):
        # Histórico: 5 partidas anteriores -> [i-5, ..., i-1]
        hist = df_team.iloc[i - WINDOW_SIZE:i].copy()

        # Reordena dentro da janela:
        # _1 = partida MAIS recente do histórico (i-1)
        # _5 = partida mais antiga (i-5)
        hist = hist.iloc[::-1].reset_index(drop=True)

        # Monta vetor de features concatenando as 5 partidas
        feats = []
        for _, row in hist.iterrows():
            feats.extend(row[base_feature_cols].tolist())

        all_windows.append(feats)

        # Linha da partida atual (rótulo e contexto)
        current_row = df_team.iloc[i]
        all_contexts.append({
            "team_id": int(current_row["team_id"]),
            "current_match_id": int(current_row["match_id"]),
            "current_date": current_row["date"],
            "league_id": int(current_row["league_id"]),
            "season": current_row["season"],
            "opponent_id_current": int(current_row["opponent_id"]),
            # resultado da partida atual (−1, 0, 1)
            "result_current": int(current_row["result_code"]),
        })


In [ ]:
# 6. Montar matriz X e DataFrame final com contexto + features

X = np.array(all_windows, dtype=float)
df_context = pd.DataFrame(all_contexts)

print("Nº de janelas geradas:", len(df_context))
print("Formato de X (janelas, features):", X.shape)
print("Esperado: somatório, por time, de max(0, n_t - WINDOW_SIZE) janelas.")

df_windows = pd.concat(
    [df_context.reset_index(drop=True),
     pd.DataFrame(X, columns=window_feature_names)],
    axis=1
)

print("df_windows_hist5:", df_windows.shape)
df_windows.head(3)


In [ ]:
# 7. Salvar a tabela de janelas no banco de dados

table_name = "team_windows_hist5"
df_windows.to_sql(table_name, conn, if_exists="replace", index=False)
print(f"Tabela '{table_name}' criada/salva no banco.")

conn.close()
print("Conexão fechada.")
